In [2]:
import pandas as pd
from urllib.request import urlopen
import json

In [3]:
varcd_cod = "0012141"
url = f"https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd={varcd_cod}&lang=PT"
response = urlopen(url)
data_json = json.loads(response.read())

In [4]:
freguesias = """
União das freguesias de Aldoar, Foz do Douro e Nevogilde
União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória
União das freguesias de Lordelo do Ouro e Massarelos
Bonfim
Campanhã
Paranhos
Ramalde
""".strip().split("\n")
freguesias

['União das freguesias de Aldoar, Foz do Douro e Nevogilde',
 'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória',
 'União das freguesias de Lordelo do Ouro e Massarelos',
 'Bonfim',
 'Campanhã',
 'Paranhos',
 'Ramalde']

In [5]:
l1 = data_json[0]["Dados"]["2021"]
l2 = filter(lambda x: x["geodsg"] in freguesias, l1)
l3 = set([(x["geocod"], x["geodsg"]) for x in l2])
l3

{('091208', 'Paranhos'),
 ('131202', 'Bonfim'),
 ('131203', 'Campanhã'),
 ('131210', 'Paranhos'),
 ('131211', 'Ramalde'),
 ('131216', 'União das freguesias de Aldoar, Foz do Douro e Nevogilde'),
 ('131217',
  'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória'),
 ('131218', 'União das freguesias de Lordelo do Ouro e Massarelos')}

In [6]:
datas_jsons = []
for code, name in l3:
    varcd_cod = '0011706'
    url = f"https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd={varcd_cod}&Dim2={code}&lang=PT"
    response = urlopen(url)
    datas_jsons.append(json.loads(response.read()))
"done"

'done'

In [7]:
final_movement_json = []
for data in datas_jsons:
    l_movement = data[0]["Dados"]['2021']
    l_movement = list(filter(lambda x: x["dim_5"] == "T" and "freguesia" in x["dim_6_t"], l_movement))
    final_movement_json += l_movement
final_movement = pd.DataFrame(final_movement_json)
final_movement = final_movement.drop(columns=["geocod","dim_3", "dim_4", "dim_5", "dim_5_t", "dim_6"])
final_movement = final_movement.rename(columns={"geodsg":"freguesia","dim_3_t":"genero", "dim_4_t":"ocupacao" ,"dim_6_t":"destino"})
final_movement.to_csv("movimentos_porto_por_freguesia.csv", index=False)